In [1]:
import lucid
import lucid.modelzoo.vision_models as models
import lucid.optvis.objectives as objectives
import requests
from PIL import Image
import io
import numpy as np
import tensorflow as tf
import lucid.optvis.render as render
import lucid.optvis.param as param
import matplotlib.pyplot as plt
import matplotlib.cm
import cv2
from sklearn.decomposition import NMF

ModuleNotFoundError: No module named 'lucid'

In [ ]:
model = models.InceptionV1()
model.load_graphdef()

# download a mapping of the imagenet class ids to text
imagenet_classes_request = requests.get("https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt")
imagenet_classes_list = [None] + [a.split(' ')[2] for a in str(imagenet_classes_request.content).split('\\n')]

# download an image from flickr
response = requests.get("http://c1.staticflickr.com/5/4070/5148597478_0c34ec0b7e_n.jpg")
image = Image.open(io.BytesIO(response.content))
image = np.array(image, dtype=np.float32)

In [ ]:
# some functions you will want to use

# compute a k-NMF of data A
def nfm(k, A):
    faktorizer = NMF(n_components=k)
    shape = A.shape
    nmf_shape = (np.prod(shape[:-1]), shape[-1])
    U = faktorizer.fit_transform(A.reshape(nmf_shape))
    V = faktorizer.components_
    U = U.reshape(shape[1:-1] + (k,) )
    V = V.reshape((k,shape[-1]) )
    return U, V

# compute the necessary forward and backward passes on a model in order to obtain
# the activations at the desicred layer, the gradients for specific logits
# and the evaluation of the logits
# activations is a tensor with the actions at layer `layer`
# grads is a list where each entry is a tensor with the gradient at layer `layer
# from the class-logit with the same position in the array classes
def foward_pass_and_gradients(model, image, classes, layer):
    with tf.Graph().as_default(), tf.Session():
        t_input = tf.placeholder_with_default(image, [None, None, 3])
        T = render.import_model(model, t_input, t_input)
        activations = T(layer).eval()
        logits = T("softmax2_pre_activation")
        grads = []
        for c in classes:
            t_grad = tf.gradients([logits[0, c]], [T(layer)])[0]   
            grad = t_grad.eval({T(layer) : activations})
            grads.append(grad)
        return activations, grads, logits.eval() 


In [ ]:
def building_blocks_of_interpretability(model, image, layer, classes, imagenet_classes_list, k=5,
                                        saliency_map_treshold=0.7, visualization_size=100, diversity=1,
                                        lambda_diversity=1e3):
    layer_names = list(map(lambda x: x.name, model.layers))
    assert 1 <= k
    assert layer in layer_names
    classes_ids = list(map(lambda x: imagenet_classes_list.index(x), classes))
    activations, gradients, logits = foward_pass_and_gradients(model, image, classes_ids, layer)
    top_classes = logits[0, :].ravel().argsort()[::-1][:10]
    print('Top 10 classes:')
    print(list(map(lambda x: imagenet_classes_list[x], top_classes.tolist())))
    # TODO implement the aglorithm here

Note: If you are running on a CPU visualzing a 60x60 image takes roughly 1 or 2 minutes on a modern laptop. So during developement/debugging you will want to keep k and the visualization_size small or move to a GPU (for example move the notebook to colab, where you did the lucid tutorials, which allows free GPU access for 2 hour sessions).

In [ ]:
layer_names = list(map(lambda x: x.name, model.layers))
print(layer_names)

In [ ]:
layer = 'mixed5a'
target_classes = ['bloodhound', 'tiger_cat']

In [ ]:
# for a good visaualization you will need to incread the parameters a lot
# but use this simple set for developement and debugging
building_blocks_of_interpretability(model, image, layer, target_classes, imagenet_classes_list,
                                    k=2, visualization_size=20, diversity=1)